# $f_{\rm det}$ Calculation

Here we want to provide the code to calculate $f_{\rm det}$ for a given set of isochrhones. Since this can be an expensive task, we're going to use Cython to speed up some parts of the code.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
%load_ext Cython

In [5]:
# First specify the properties of the populations you'd like to calculate f_detect for
# by giving lists of metallicities and ages. These should be linearly spaced 
metlist = np.array([0.5,0.25,0.0,-0.25,-0.5,-0.75,-1.0,-1.25,-1.5,-1.75,-2.0,-2.25,-2.5,-2.75,-3.,-3.25])
agelist = np.array([8.95,9.,9.05,9.1,9.15,9.2,9.25,9.3,9.35,9.4,9.45,9.5,9.55,9.6,9.65,9.7,9.75,9.8,9.85,9.9,9.95,10.,10.05,10.1,10.15])
nmet = len(metlist) # number of metallicities
nage = len(agelist) # number of ages
dage = agelist[1] - agelist[0] # logarithmic difference in age bins, for selection below

## specify directory where isochrones are stored, this is set up for the 
## MIST Roman Isocrones, changing that would involve changing the below
iso_dir = "../mist_isos/"
## name od output file
output_name = "data/fdet_mist" # it will have a .npy at the end

## define the IMF normalization
## specify the power-law index for the IMF that you would like to use
mpow = -1.3
## specify the number of mass samples you would like for integrals to be 
## numericall performed below
nmass_samp = 100000000
# mass range in solar masses
(mMin,mMax) = (0.1,100)
# masses spaced linearly throughout imf space
mlin = np.linspace(mMin,mMax,nmass_samp)
# the IMF normalization over the mass range of interest
xim_norm = np.trapz(mlin**mpow,mlin)

# then specify the range of *absolute* magnitudes you would like to cover
# these are related to the exposure time of the observation and the distance 
# to the observed population through Eq 2 of our paper
nmags = 100
# what are the brightest and faintest magnitudes you want to calculate for
(minMag,maxMag) = (3,-7)
# get the range of absolute magnitudes you would like to calculate fdet for
Mags = np.linspace(minMag,maxMag,nmags)
# specify the number of filters. The default is 6 (Z,Y,J,H,F). Changing this 
# number will require some hard code changes below
nfilts = 5

In [3]:
%%cython
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True
import numpy as np
cimport numpy as np


cimport cython

cpdef double get_f(const double[::1] B_mu, const double B_cut, 
                   const double[::1] ms, const double[::1] xiofm, 
                   const int N):
    cdef:
        double fsum = 0
        double aB,bB,aX,bX,mcross,xicross
        int i
    
    for i in range(N-1):
        if((B_mu[i]<B_cut) and (B_mu[i+1] < B_cut)):
            fsum += (xiofm[i] + xiofm[i+1])*(ms[i+1] - ms[i])/2.
        
        elif((B_mu[i]<B_cut) or (B_mu[i+1] < B_cut)):
            aB = (B_mu[i+1] - B_mu[i])/(ms[i+1] - ms[i])
            bB = B_mu[i] - ms[i]*aB
            mcross = (B_cut - bB)/aB
            aX = (xiofm[i+1] - xiofm[i])/(ms[i+1] - ms[i])
            bX = xiofm[i] - ms[i]*aX
            xicross = mcross*aX + bX
            if(B_mu[i]<B_cut):
                fsum += (xiofm[i] + xicross)*(mcross - ms[i])/2.
            else:
                fsum += (xicross + xiofm[i+1])*(ms[i+1] - mcross)/2.
    return fsum

In [4]:
%%time
# shape of the total output array
totdat = np.zeros((nmet,nage,nmags,nfilts+1))
for (l,met) in enumerate(metlist):
    print(met)
    if met>-0.1:
        (age,imass,T,L) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (Z,Y,J,H,F) = np.loadtxt("%smist_fehp%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[10,11,12,14,15]).T
    else:
        (age,imass,T,L) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[1,2,4,6]).T
        (Z,Y,J,H,F) = np.loadtxt("%smist_fehm%1.2f.iso.cmd"%(iso_dir,abs(met)),usecols=[10,11,12,14,15]).T

    tsave = np.zeros((nage,nmags,nfilts + 1))
    # select a single isochrone
    for (k,a) in enumerate(agelist):
        sids = np.intersect1d(np.where(age>a-dage/2),np.where(age<a+dage/2))

        # interpolate what the right magnitudes are
        AB_Vega = np.array([0.487, 0.653, 0.958, 1.287, 1.552]) # This is m_AB - m_Vega
        Z_out = Z[sids] + AB_Vega[0]
        Y_out = Y[sids] + AB_Vega[1]
        J_out = J[sids] + AB_Vega[2]
        H_out = H[sids] + AB_Vega[3]
        F_out = F[sids] + AB_Vega[4]
        mlin = imass[sids]
        xim = (imass[sids]**mpow)/xim_norm
        (fZdetect,fYdetect,fJdetect,fHdetect,fFdetect) = (np.zeros(nmags),np.zeros(nmags),np.zeros(nmags),np.zeros(nmags),np.zeros(nmags))
        for (i,mag) in enumerate(Mags):
            fZdetect[i] = get_f(Z_out,mag,mlin,xim,len(xim))
            fYdetect[i] = get_f(Y_out,mag,mlin,xim,len(xim))
            fJdetect[i] = get_f(J_out,mag,mlin,xim,len(xim))
            fHdetect[i] = get_f(H_out,mag,mlin,xim,len(xim))
            fFdetect[i] = get_f(F_out,mag,mlin,xim,len(xim))
        tsave[k] = np.array([Mags,fZdetect,fYdetect,fJdetect,fHdetect,fFdetect]).T
    totdat[l] = tsave

0.5
0.25
0.0
-0.25
-0.5
-0.75
-1.0
-1.25
-1.5
-1.75
-2.0
-2.25
-2.5
-2.75
-3.0
-3.25
CPU times: user 36.9 s, sys: 1.3 s, total: 38.2 s
Wall time: 38.5 s


In [6]:
np.save(output_name,totdat)